In [1]:
import sys
sys.path[:0] = ['/usr/share/inkscape/extensions']
import inkex
import re

In [2]:
class LayerFilter(inkex.Effect):
    pass

In [3]:
ns = 'http://www.inkscape.org/namespaces/inkscape'
groupmode_tag = '{%s}groupmode' %(ns,)
id_tag = 'id'
label_tag = '{%s}label' % (ns,)
style_tag = 'style'

In [4]:
exports_file = 'svgexports'
def read_exports():
    f = open(exports_file,'r')
    for line in f:
        line = line.strip()
        if line == '':
            continue
        if line.startswith('#'):
            continue
        fields = re.split(r'\t+',line)
        yield fields
    f.close()

In [8]:
prev_svg = None
prev_layers = []
for svgfile,ids,targetbase in read_exports():
    if svgfile == prev_svg:
        Layers = prev_layers
    else:
        Layers = []
        prev_svg = svgfile
    if ids.startswith('+'):
        ids = ids[1:]
    else:
        Layers = []
    Layers = Layers + [x for x in ids.split(',') if x.strip() != '']
    prev_layers = Layers
    print(svgfile, Layers, targetbase)
    dst = targetbase + '.svg'
    if svgfile == dst:
        raise Exception('Will not overwrite source file.')
    lf = LayerFilter()
    lf.parse(svgfile)
    root = lf.document.getroot()
    for el in root.getchildren():
        if el.get(groupmode_tag) == 'layer':
            s = 'display:inline' if el.get(label_tag) in Layers else 'display:none'
            el.set(style_tag,s)
            print('  ',el, el.get(label_tag),el.get(id_tag),el.get(style_tag))
    
    lf.document.write(dst)

hextruss.svg ['Layer 1'] hextruss-1
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84f1648> step4 layer6 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84f16c8> step3 layer5 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84f1708> step2 layer4 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84f1b08> step1 layer3 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84f1b48> Paths layer2 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84f1688> Layer 1 layer1 display:inline
hextruss.svg ['Layer 1', 'Paths'] hextruss-2
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84e6e88> step4 layer6 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84e6b08> step3 layer5 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84e6c88> step2 layer4 display:none
   <Element {http://www.w3.org/2000/svg}g at 0x7faef84e6688> step1 layer3 display:none
   <Element {http://www.w3.org/2000/svg}g at 